In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import optuna

In [6]:
input = pd.read_csv('final_df_start_center_finish.csv')

In [8]:
pred_time = pd.read_csv('pred_time.csv', index_col="Id")
input["pred_time"] = pred_time

In [18]:
# input = input.drop(['running_time', 'completed_time'], axis=1)
input = input.fillna(0)

In [56]:
y_train = input['delta_time']
X_train = input.drop(['delta_time'], axis=1)

In [57]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [61]:
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)

scaler_y = MinMaxScaler()

y_train = scaler_y.fit_transform(y_train.values.reshape(-1, 1))

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.1, random_state = 42)

In [71]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [80]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, features, labels):
        self.labels = labels
        self.features = features

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        return np.array(self.labels[idx])

    def get_batch_features(self, idx):
        return np.array(self.features[idx])

    def __getitem__(self, idx):
        batch_features = self.get_batch_features(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_features, batch_y

In [81]:
X_test.shape

torch.Size([500, 116])

In [82]:
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)

C:\Users\Andrii\AppData\Local\Temp/ipykernel_1724/3402302031.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train, dtype=torch.float32)
C:\Users\Andrii\AppData\Local\Temp/ipykernel_1724/3402302031.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
C:\Users\Andrii\AppData\Local\Temp/ipykernel_1724/3402302031.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test, dtype=torch.float32)
C:\Users\Andrii\AppData\Local\Tem

In [68]:
y_train.shape

torch.Size([4497, 1])

In [169]:
class Block(nn.Module):
    def __init__(self, in_features, out_features, dropout_rate=0.2, dropout_rate_2=0.2, use_last_batchnorm=True):
        """
        Args:
          in_channels (int):  Number of input channels.
          out_channels (int): Number of output channels.
          stride (int):       Controls the stride.
        """
        super(Block, self).__init__()

        self.block = nn.Sequential(
            nn.BatchNorm1d(in_features),
            nn.Linear(in_features, in_features),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(in_features, in_features),
            nn.Dropout(dropout_rate_2))
        
        modules = []
        
        if use_last_batchnorm:
            modules.append(nn.BatchNorm1d(in_features))
        modules.append(nn.ReLU())            
        modules.append(nn.Linear(in_features, out_features))
        self.part2 = nn.Sequential(*modules)

            
    def forward(self, x):
        out = self.block(x)

        out += x
        
        out = self.part2(out)

        return out

In [175]:
# Build a model by implementing define-by-run design from Optuna
def build_model_custom(trial):
    
    n_layers = trial.suggest_int("n_layers", 1, 8)
    layers = []

    in_features = X_train.shape[1]

    out_features = trial.suggest_int("n_units_l-1", 64, 512)

    dropout_1 = trial.suggest_uniform('dropout1', 0, 0.5)
    dropout_2 = trial.suggest_uniform('dropout2', 0, 0.5)

    layers.append(nn.Linear(in_features, out_features))

    in_features = out_features
    
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 64, 512)
        
        layers.append(Block(in_features, out_features, dropout_1, dropout_2, i != n_layers - 1))
        in_features = out_features
        
    layers.append(nn.Linear(in_features, 1))
    layers.append(nn.ReLU())
    
    return nn.Sequential(*layers)

In [176]:
# Train and evaluate the accuracy of neural network with the addition of pruning mechanism
def train_and_evaluate(param, model, trial = None):

    train, val = Dataset(X_train, y_train), Dataset(X_test, y_test)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=param['batch_size'], shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=param['batch_size'])

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.MSELoss()
    optimizer = getattr(optim, param['optimizer'])(model.parameters(), lr=param['learning_rate'], weight_decay=param['weight_decay'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()

    for epoch_num in range(param['max_epochs']):

            total_rmse_train = 0
            total_loss_train = 0

            for train_input, train_label in train_dataloader:

                train_label = train_label.to(device)
                train_input = train_input.to(device)

                output = model(train_input.float())
                
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                
                rmse = mean_squared_error(train_label.cpu(), output.cpu().detach().numpy(), squared=False)
                total_rmse_train += rmse

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_rmse_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    val_input = val_input.to(device)

                    output = model(val_input.float())

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                    
                    rmse =  mean_squared_error(val_label.cpu(), output.cpu().detach().numpy(), squared=False)
                    total_rmse_val += rmse
            
            rmse = total_rmse_val/len(y_test)
            
            scheduler.step(rmse)

            if trial:
                # Add prune mechanism
                trial.report(rmse, epoch_num)

                if trial.should_prune():
                    raise optuna.exceptions.TrialPruned()

    return rmse

In [177]:
# Define a set of hyperparameter values, build the model, train the model, and evaluate the accuracy
def objective(trial):

     params = {
              'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
              'weight_decay': trial.suggest_loguniform('weight_decay', 1e-6, 1e-3),
              'optimizer': trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
              'batch_size': trial.suggest_categorical("batch_size", [32, 64, 128, 256]),
               'max_epochs': trial.suggest_categorical("max_epochs", [30, 40, 50, 60, 70, 100]),
              }
    
     model = build_model_custom(trial)

     rmse = train_and_evaluate(params, model, trial)

     return rmse

In [201]:
# study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=25, show_progress_bar=True)

C:\Users\Andrii\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\optuna\progress_bar.py:47: ExperimentalWarning:

Progress bar is experimental (supported from v1.2.0). The interface can change in the future.



  0%|          | 0/25 [00:00<?, ?it/s]

[I 2023-03-05 05:49:40,318] Trial 55 pruned. 
[I 2023-03-05 05:49:40,721] Trial 56 pruned. 
[I 2023-03-05 05:49:41,076] Trial 57 pruned. 
[I 2023-03-05 05:49:41,690] Trial 58 pruned. 
[I 2023-03-05 05:49:42,257] Trial 59 pruned. 
[I 2023-03-05 05:49:42,559] Trial 60 pruned. 
[I 2023-03-05 05:49:43,020] Trial 61 pruned. 
[I 2023-03-05 05:49:43,401] Trial 62 pruned. 
[I 2023-03-05 05:50:04,148] Trial 63 finished with value: 0.0005483960807323455 and parameters: {'learning_rate': 0.0016103785180026265, 'weight_decay': 0.0004311699984173278, 'optimizer': 'Adam', 'batch_size': 256, 'max_epochs': 60, 'n_layers': 3, 'n_units_l-1': 128, 'dropout1': 0.4577813340209889, 'dropout2': 0.27515869087232503, 'n_units_l0': 170, 'n_units_l1': 227, 'n_units_l2': 275}. Best is trial 45 with value: 0.0005366033911705017.
[I 2023-03-05 05:50:05,864] Trial 64 pruned. 
[I 2023-03-05 05:50:06,883] Trial 65 pruned. 
[I 2023-03-05 05:50:07,125] Trial 66 pruned. 
[I 2023-03-05 05:50:07,368] Trial 67 pruned. 
[I 2

In [202]:
study.best_trial.params

{'learning_rate': 0.003320666903799562,
 'weight_decay': 0.0002862478870592661,
 'optimizer': 'Adam',
 'batch_size': 256,
 'max_epochs': 100,
 'n_layers': 1,
 'n_units_l-1': 147,
 'dropout1': 0.4586327591625059,
 'dropout2': 0.07180085102853859,
 'n_units_l0': 145}

In [203]:
optuna.visualization.plot_intermediate_values(study)

In [204]:
n_layers = study.best_trial.params["n_layers"]
layers = []

in_features = X_train.shape[1]

out_features = study.best_trial.params["n_units_l-1"]

dropout_1 = study.best_trial.params['dropout1']
dropout_2 = study.best_trial.params['dropout2']

layers.append(nn.Linear(in_features, out_features))

in_features = out_features

for i in range(n_layers):
    out_features = study.best_trial.params["n_units_l{}".format(i)]
    
    layers.append(Block(in_features, out_features, dropout_1, dropout_2, i != n_layers - 1))
    in_features = out_features
    
layers.append(nn.Linear(in_features, 1))
layers.append(nn.ReLU())

model = nn.Sequential(*layers)


rmse = train_and_evaluate(study.best_trial.params, model)

train, val = Dataset(X_train, y_train), Dataset(X_test, y_test)

val_dataloader = torch.utils.data.DataLoader(val, batch_size=study.best_trial.params['batch_size'])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

result = []

total_rmse_val = 0

for val_input, val_label in val_dataloader:

    val_label = val_label.to(device)
    val_input = val_input.to(device)

    output = model(val_input.float())

    result.append(output.cpu().detach().numpy())

    rmse =  mean_squared_error(val_label.cpu(), output.cpu().detach().numpy(), squared=False)
    total_rmse_val += rmse

rmse = total_rmse_val/len(y_test)

print(rmse)

y_res = np.concatenate(result)

0.0005564707517623902


In [205]:
y_res_denormalized = scaler_y.inverse_transform(y_res)
y_test_denormalized = scaler_y.inverse_transform(y_test.detach().numpy())

In [206]:
mean_squared_error(y_res_denormalized, y_test_denormalized, squared=False)

132.00104